# Batch Calculation of the Classification Accuracy of ROI Brain Regions

This tutorial demonstrates a complete workflow for decoding fMRI responses associated with free-recall behavior. Using ROI-level activation patterns from multiple cortical regions, we build logistic regression models to predict whether each target image was successfully recalled. The tutorial covers data preprocessing, feature selection using statistical tests, hyperparameter tuning, and cross-validated evaluation. We compare performance using all voxels versus the top statistically informative voxels, and assess significance relative to a shuffled-label baseline. This end-to-end example provides a practical template for fMRI classification analysis and can be easily extended to other cognitive or neuroimaging tasks.

## 1. Package Import

In [1]:
import numpy as np
import pandas as pd
import os
from scipy.stats import ttest_ind, t
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

## 2. Parameter Setup

In [ ]:
CSV_PATH = '/BCAI-fmri/free_recall_order.csv'
CORT_DIR = '/BCAI-fmri/features/cort'

TARGET_IDS = [39, 16, 30, 59, 87, 26]              # The six target picture IDs
TOP_N_VOXELS = 10                                   # Number of voxels selected based on t-statistics
C_PARAMS = [100, 50, 20, 10, 5, 1, 0.5, 0.2, 0.1,
            0.05, 0.01]                              # Regularization strengths for logistic regression
RANDOM_STATE = 42
CV_FOLDS = 5                                         # 5-fold cross-validation

## 3. Key Function Define

In [ ]:

def select_top10_voxels(feats, labels):
    """
    Select the top N voxels using two-sample t-test between labels 0 and 1.
    The absolute t-value is used for ranking.
    """
    results = []
    for voxel_idx in range(feats.shape[1]):
        group0 = feats[labels == 0, voxel_idx][~np.isnan(feats[labels == 0, voxel_idx])]
        group1 = feats[labels == 1, voxel_idx][~np.isnan(feats[labels == 1, voxel_idx])]
        
        # Ensure sufficient samples
        if len(group0) < 5 or len(group1) < 5:
            continue
        
        t_val, p_val = ttest_ind(group0, group1, equal_var=False, nan_policy='omit')
        results.append([voxel_idx, abs(t_val), p_val])

    df = pd.DataFrame(results, columns=['voxel_idx', 't_abs', 'p_val']).sort_values('t_abs', ascending=False)
    return df.head(TOP_N_VOXELS)['voxel_idx'].tolist()


def tune_lr_with_cv(X, y, c_params):
    """
    Tune logistic regression using multiple C values and evaluate using cross-validation.
    Returns: mean accuracy, best C, 95% CI, significance (p < 0.05 vs. random baseline).
    """
    # Split data for hyperparameter tuning
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Find best C
    best_acc = 0
    best_c = c_params[0]
    
    for c in c_params:
        lr = LogisticRegression(
            C=c, penalty='l2', max_iter=1000, class_weight='balanced',
            random_state=RANDOM_STATE
        )
        lr.fit(X_train_scaled, y_train)
        acc = accuracy_score(y_test, lr.predict(X_test_scaled))
        
        if acc > best_acc:
            best_acc = acc
            best_c = c

    # Cross-validation using the best C
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    X_scaled = scaler.fit_transform(X)
    
    cv_scores = cross_val_score(
        LogisticRegression(
            C=best_c, penalty='l2', max_iter=1000,
            class_weight='balanced', random_state=RANDOM_STATE
        ),
        X_scaled, y, cv=cv, scoring='accuracy'
    )
    
    # Compute 95% Confidence Interval
    mean_cv_acc = cv_scores.mean()
    sem = np.std(cv_scores) / np.sqrt(len(cv_scores))
    ci95 = t.interval(0.95, df=len(cv_scores) - 1, loc=mean_cv_acc, scale=sem)

    # Significance test vs. random 0.5 accuracy
    t_stat, p_val = ttest_ind(cv_scores, [0.5] * len(cv_scores), alternative='greater')
    is_significant = p_val < 0.05
    
    return mean_cv_acc, best_c, ci95, is_significant


def get_random_baseline(X, y, best_c):
    """
    Generate a shuffled-label baseline to evaluate whether the classifier
    performs above random chance.
    """
    y_shuffled = np.random.permutation(y)
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    X_scaled = StandardScaler().fit_transform(X)

    random_cv_scores = cross_val_score(
        LogisticRegression(
            C=best_c, penalty='l2', max_iter=1000,
            class_weight='balanced', random_state=RANDOM_STATE
        ),
        X_scaled, y_shuffled, cv=cv, scoring='accuracy'
    )
    return random_cv_scores.mean()

## 4. Batch Processing Across Brain Regions

In [ ]:
df_labels = pd.read_csv(CSV_PATH)
mask_target = df_labels['pictureID'].isin(TARGET_IDS)
labels_all = df_labels.loc[mask_target, 'recall'].values

results_list = []
npy_files = [f for f in os.listdir(CORT_DIR)
             if f.endswith('.npy') and f.startswith('beta_')]

print(f"Starting batch analysis (full voxels + Top {TOP_N_VOXELS} voxels) across {len(npy_files)} brain regions...")

for npy_file in npy_files:
    
    # Load fMRI features
    brain_region_path = os.path.join(CORT_DIR, npy_file)
    data = np.load(brain_region_path)
    
    feats = data[mask_target]
    if len(feats) != len(labels_all):
        continue
    
    feats_imputed = SimpleImputer(strategy='mean').fit_transform(feats)

    # ------------------ Full Voxel Logistic Regression ------------------
    mean_acc_full, best_c_full, ci95_full, is_significant_full = tune_lr_with_cv(
        feats_imputed, labels_all, C_PARAMS
    )
    random_acc_full = get_random_baseline(feats_imputed, labels_all, best_c_full)

    # ------------------ Top 10 Voxel Logistic Regression ------------------
    top_voxels = select_top10_voxels(feats_imputed, labels_all)
    
    if len(top_voxels) < TOP_N_VOXELS:
        mean_acc_top = np.nan
        best_c_top = np.nan
        ci95_top = (np.nan, np.nan)
        is_significant_top = 'N/A'
        random_acc_top = np.nan

    else:
        X_top = feats_imputed[:, top_voxels]
        mean_acc_top, best_c_top, ci95_top, is_significant_top = tune_lr_with_cv(
            X_top, labels_all, C_PARAMS
        )
        random_acc_top = get_random_baseline(X_top, labels_all, best_c_top)

    # Collect results
    brain_region_name = npy_file.replace('beta_', '').replace('.npy', '')
    
    results_list.append({
        'Region': brain_region_name,

        # Full voxel results
        'FullVoxels-BestC': best_c_full,
        'FullVoxels-Accuracy': round(mean_acc_full, 3),
        'FullVoxels-RandomAcc': round(random_acc_full, 3),
        'FullVoxels-95CI': f"[{round(ci95_full[0], 3)}, {round(ci95_full[1], 3)}]",
        'FullVoxels-Significant': 'Yes' if is_significant_full else 'No',

        # Top voxel results
        'TopVoxels-BestC': best_c_top if not np.isnan(best_c_top) else 'N/A',
        'TopVoxels-Accuracy': round(mean_acc_top, 3) if not np.isnan(mean_acc_top) else 'N/A',
        'TopVoxels-RandomAcc': round(random_acc_top, 3) if not np.isnan(random_acc_top) else 'N/A',
        'TopVoxels-95CI': f"[{round(ci95_top[0], 3)}, {round(ci95_top[1], 3)}]"
                          if not np.isnan(ci95_top[0]) else 'N/A',
        'TopVoxels-Significant': (
            'Yes' if is_significant_top is True else
            'No' if is_significant_top is False else 'N/A'
        ),
    })

## 5. Output Table

In [ ]:
results_df = pd.DataFrame(results_list).sort_values(
    'TopVoxels-Accuracy', ascending=False, na_position='last'
)

print("\n" + "="*180)
print("Batch Analysis Results Across Brain Regions (Full Voxels vs. Top Voxels)")
print("="*180)
print(results_df.to_string(index=False))

print("\n" + "="*180)
print(f"Top {TOP_N_VOXELS} Voxels — Top 10 Regions by Accuracy")
print("="*180)
top10_brains = results_df[results_df['TopVoxels-Accuracy'] != 'N/A'].head(10)
print(top10_brains.to_string(index=False))
print("="*180)